## Reading graphs

In this exercise, before you compute projections, you're going to practice working with one of NetworkX's disk I/O functions, read_edgelist(). read_edgelist() creates a graph from the edgelist file. The graph that you'll be working with is a bipartite graph describing the American Revolution. There are two node partitions - 'people' and 'clubs', and edges denote a person being a member of a club.

### Instructions
    - Import networkx as nx.
    - Use nx.read_edgelist() to read in 'american-revolution.edgelist'.
    - In the dataset, 'clubs' do not have a . symbol in their node name. Use this information to assign nodes to 'clubs' or 'people' partitions. Remember the 'bipartite' keyword!
    - Print the edges of the graph.

In [ ]:
# Import networkx
import networkx as nx

# Read in the data: g
G = nx.read_edgelist('american-revolution.edgelist')

# Assign nodes to 'clubs' or 'people' partitions
for n, d in G.nodes(data=True):
    if '.' in n:
        G.nodes[n]['bipartite'] = 'people'
    else:
        G.nodes[n]['bipartite'] = 'clubs'
        
# Print the edges of the graph
print(G.edges())


## Computing projection

It's now time to try your hand at computing the projection of a bipartite graph to the nodes on one of its partitions. This will help you gain practice with converting between a bipartite version of a graph and its unipartite projections. Remember from the video that the "projection" of a graph onto one of its partitions is the connectivity of the nodes in that partition conditioned on connections to nodes on the other partition. Made more concretely, you can think of the "connectivity of customers based on shared purchases".

To help you get started, here's a hint on list comprehensions. List comprehensions can include conditions, so if you want to filter a graph for a certain type of node, you can do: <code>[n for n, d in G.nodes(data=True) if d['key'] == 'some_value']</code>.

### Instructions
    - Prepare the people nodelist using a list comprehension. If the 'bipartite' keyword of a node n in G equals 'people', then that node should be part of the nodelist.
    - Prepare the clubs nodelist by iterating over the nodes of G, including the metadata. Here, note that you have to check if the 'bipartite' keyword of the metadata dictionary d equals 'clubs'. Note: This is simply an alternate way of creating the nodelist. You do not have to iterate over the metadata - you can follow the same approach you used to create the people nodelist, simply checking for 'clubs' instead. We're asking you to use the other approach here so you get practice with both.
    - Use nx.bipartite.projected_graph() to compute the people and clubs projections. Store the results as peopleG and clubsG.
        - This function takes in two arguments: The graph G, and the nodelist.

In [ ]:
# Prepare the nodelists needed for computing projections: people, clubs
# This exercise shows you two ways to do it, one with `data=True` and one without.
people = [n for n in G.nodes() if G.nodes[n]['bipartite'] == 'people']
clubs = [n for n, d in G.nodes(data=True) if d['bipartite'] == 'clubs']

# Compute the people and clubs projections: peopleG, clubsG
peopleG = nx.bipartite.projected_graph(G, people)
clubsG = nx.bipartite.projected_graph(G, clubs)

## Plot degree centrality on projection

Here, you're going to compare the degree centrality distributions for each of the following graphs: the original graph G, the people graph projection peopleG, and the clubs graph projection clubsG. This will reinforce the difference in degree centrality score computation between bipartite and unipartite versions of degree centrality metrics. The node lists people and clubs have been pre-loaded for you.

Recall from the video that the bipartite functions require passing in a container of nodes, but will return all degree centrality scores nonetheless. Remember also that degree centrality scores are stored as dictionaries (mapping node to score).

### Instructions
    - Plot the degree centrality distribution of the original graph G, using the degree_centrality function from the bipartite module: nx.bipartite.degree_centrality(). It takes in two arguments: The graph G, and one of the node lists (people or clubs).
    - Plot the degree centrality distribution of the peopleG graph, using the normal/non-bipartite degree_centrality function from NetworkX: nx.degree_centrality().
    - Plot the degree centrality distribution of the clubsG graph, using the normal/non-bipartite degree_centrality function from NetworkX: nx.degree_centrality().

In [ ]:
import matplotlib.pyplot as plt

# Plot the degree centrality distribution of both node partitions from the original graph
plt.figure() 
original_dc = nx.bipartite.degree_centrality(G, people)  
# Remember that you can cast a dictionary values to a list.
plt.hist(original_dc.values(), alpha=0.5)
plt.yscale('log')
plt.title('Bipartite degree centrality')
plt.show()


# Plot the degree centrality distribution of the peopleG graph
plt.figure()
people_dc = nx.degree_centrality(peopleG)
plt.hist(people_dc.values())
plt.yscale('log')
plt.title('Degree centrality of people partition')
plt.show()

# Plot the degree centrality distribution of the clubsG graph
plt.figure()
clubs_dc = nx.degree_centrality(clubsG)
plt.hist(clubs_dc.values())
plt.yscale('log')
plt.title('Degree centrality of clubs partition')
plt.show()

## Compute adjacency matrix

Now, you'll get some practice using matrices and sparse matrix multiplication to compute projections! In this exercise, you'll use the matrix multiplication operator @ that was introduced in Python 3.5.

You'll continue working with the American Revolution graph. The two partitions of interest here are 'people' and 'clubs'.

### Instructions
    - Get the list of people and list of clubs from the graph G using the get_nodes_from_partition() function that you defined in the previous chapter. This function accepts two parameters: A graph, and a partition.
    - Compute the biadjacency matrix using nx.bipartite.biadjacency_matrix(), setting the row_order parameter to people_nodes and the column_order parameter to clubs_nodes. Remember to also pass in the graph G.
    - Compute the user-user projection by multiplying (with the @ operator) the biadjacency matrix bi_matrix by its transposition, bi_matrix.T.

In [ ]:
# Get the list of people and list of clubs from the graph: people_nodes, clubs_nodes
people_nodes = get_nodes_from_partition(G, 'people')
clubs_nodes = get_nodes_from_partition(G, 'clubs')

# Compute the biadjacency matrix: bi_matrix
bi_matrix = nx.bipartite.biadjacency_matrix(G, row_order=people_nodes, column_order=clubs_nodes)

# Compute the user-user projection: user_matrix
user_matrix = bi_matrix @ bi_matrix.T

print(user_matrix)

## Find shared membership: Transposition

As you may have observed, you lose the metadata from a graph when you go to a sparse matrix representation. You're now going to learn how to impute the metadata back so that you can learn more about shared membership.

The user_matrix you computed in the previous exercise has been preloaded into your workspace.

Here, the np.where() function will prove useful. This is what it does: given an array, say, a = [1, 5, 9, 5], if you want to get the indices where the value is equal to 5, you can use <code>idxs = np.where(a == 5)</code>. This gives you back an array in a tuple, (array([1, 3]),). To access those indices, you would want to index into the tuple as idxs[0].

### Instructions
    - Find out the names of people who were members of the most number of clubs.
        - To do this, first compute diag by using the .diagonal() method on user_matrix.
        - Then, using np.where(), select those indices where diag equals diag.max(). This returns a tuple: Make sure you access the relevant indices by indexing into the tuple with [0].
        - Iterate over indices and print out each index i of people_nodes using the provided print() function.
    - Set the diagonal to zero and convert it to a "coordinate matrix format". This code has been provided for you in the answer.
    - Find pairs of users who shared membership in the most number of clubs.
        - Using np.where(), access the indices where users_coo.data equals users_coo.data.max().
        - Iterate over indices2 and print out each index idx of people_node's users_coo.row and users_coo.col.

In [ ]:
import numpy as np

# Find out the names of people who were members of the most number of clubs
diag = user_matrix.diagonal()
indices = np.where(diag == diag.max())[0]  
print('Number of clubs: {0}'.format(diag.max()))
print('People with the most number of memberships:')
for i in indices:
    print('- {0}'.format(people_nodes[i]))

# Set the diagonal to zero and convert it to a coordinate matrix format
user_matrix.setdiag(0)
users_coo = user_matrix.tocoo()

# Find pairs of users who shared membership in the most number of clubs
indices2 = np.where(users_coo.data == users_coo.data.max())[0]
print('People with most number of shared memberships:')
for idx in indices2:
    print('- {0}, {1}'.format(people_nodes[users_coo.row[idx]], people_nodes[users_coo.col[idx]]))  

## Make nodelist

You're now going to practice converting graphs to pandas representation. If you have taken any of DataCamp's pandas courses, you will know that there is a DataFrame.to_csv('filename.csv') method that lets you save it as a CSV file, which is a human-readable version. The main concept we hope you take away from here is the process of converting a graph to a list of records.

Start by re-familiarizing yourself with the graph data structure by calling list(G.nodes(data=True))[0] in the IPython Shell to examine one node in the graph.

### Instructions
    - Initialize an empty node list called nodelist.
    - Use a for loop to iterate over the nodes of the G_people. Inside the loop:
        - Update the nodeinfo dictionary using the .update() method with d as the argument.
        - Append the nodeinfo dictionary to nodelist.
    - Create a pandas DataFrame of the nodelist called node_df using the pd.DataFrame() function.

In [ ]:
# Initialize a list to store each edge as a record: nodelist
nodelist = []
for n, d in G_people.nodes(data=True):
    # nodeinfo stores one "record" of data as a dict
    nodeinfo = {'person': n} 
    
    # Update the nodeinfo dictionary 
    nodeinfo.update(d)
    
    # Append the nodeinfo to the node list
    nodelist.append(nodeinfo)
    

# Create a pandas DataFrame of the nodelist: node_df
node_df = pd.DataFrame(nodelist)
print(node_df.head())

## Make edgelist

Now, you're going to apply the same ideas to making an edge list. Go forth and give it a shot!

As with the previous exercise, run list(G.edges(data=True))[0] in the IPython Shell to get a feel for the edge list data structure before proceeding.

### Instructions
    - Initialize a list called edgelist to store each edge as a record.
    - Use a for loop to iterate over the edges of G_people. Inside the loop:
        - Initialize a dictionary called edgeinfo that shows edge information.
        - Update the edgeinfo dictionary with the metadata dictionary d.
        - Append the edgeinfo dictionary to edgelist.
    - Create a pandas DataFrame of the edgelist called edge_df.

In [ ]:
# Initialize a list to store each edge as a record: edgelist
edgelist = []
for n1, n2, d in G_people.edges(data=True):
    # Initialize a dictionary that shows edge information: edgeinfo
    edgeinfo = {'node1':n1, 'node2':n2}
    
    # Update the edgeinfo data with the edge metadata
    edgeinfo.update(d)
    
    # Append the edgeinfo to the edgelist
    edgelist.append(edgeinfo)
    
# Create a pandas DataFrame of the edgelist: edge_df
edge_df = pd.DataFrame(edgelist)
print(edge_df.head())